## BaggingClassifier + TF-IDF 

In [52]:
import pandas as pd
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [53]:
training_data = pd.read_csv('train.csv')

#### Combining the 3 columns ( keyword + location + text ) - filling the NAN by a blank or ''

In [54]:
training_data['text'] = training_data['keyword'].fillna('') + training_data['location'].fillna('')  \
                        + training_data['text'].fillna('')
training_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [55]:
training_data = training_data.drop(columns=['id','keyword','location'],axis=1)
training_data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


#### Data Cleaning step involves tokenizing, removing the stopwords and removing non-alphanumeric tokens

In [56]:
def clean_data(text):
    tokens = text.split()
    no_stopwords = [x for x in tokens if x not in stop_words]
    no_nonalphanum = [x.lower() for x in no_stopwords if x.isalnum()]
    return ' '.join(no_nonalphanum)

In [57]:
training_data['text'] = training_data['text'].apply(clean_data)
print(training_data.shape)
training_data.head()

(7613, 2)


,text,target
0,our deeds reason may allah forgive us,1
1,forest fire near la ronge canada,1
2,all residents asked notified no evacuation she...,1
3,people receive evacuation orders california,1
4,just got sent photo ruby smoke pours school,1


In [59]:
test_data = pd.read_csv('test.csv')
test_id = test_data['id']
test_data['text'] = test_data['keyword'].fillna('') + test_data['location'].fillna('')  \
                        + test_data['text'].fillna('')
test_data = test_data.drop(columns=['id','keyword','location'],axis=1)
test_data['text'] = test_data['text'].apply(clean_data)
print(test_data.shape)
test_data.head()

(3263, 1)


,text
0,just happened terrible car crash
1,heard different stay safe
2,forest fire spot geese fleeing across i cannot...
3,apocalypse
4,typhoon soudelor kills 28 china taiwan


#### combining the training and testing vocabulary to fit the TF-IDF Vectorizer over all the available data 

In [60]:
combined_data = pd.DataFrame()
combined_data = combined_data.append(training_data,ignore_index=True,sort=False)
combined_data = combined_data.append(test_data,ignore_index=True,sort=False)
combined_data.shape

(10876, 2)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

#### fitting the TF-IDF Vectorizer with the combined data and then using it to transform the training data

In [62]:
tfidf = TfidfVectorizer()
tfidf.fit(combined_data.text)
text_tfidf = tfidf.transform(training_data.text)

In [63]:
x_train,x_test,y_train,y_test = train_test_split(text_tfidf,training_data.target)

#### Using sklearn BaggingClassifier

In [64]:
bag = BaggingClassifier()
bag.fit(x_train,y_train)
predict = bag.predict(x_test)
accuracy_score(y_test,predict)

0.7647058823529411

#### Training or fitting the BaggingClassifier with all the training data

In [65]:
bag.fit(text_tfidf,training_data.target)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)

#### Transforming the test data to vectors

In [66]:
test_tfidf = tfidf.transform(test_data.text)

#### Performing the predictions over the test data

In [67]:
predict = bag.predict(test_tfidf)

In [69]:
predictions = pd.DataFrame({'id':test_id,'target':predict})
predictions.to_csv('submission.csv',index=False)

#### Accuracy :---
#####                - Local Accuracy : 76.47 (over split training data)
#####                - Online Accuracy : 77.81 (After fitting over all training data)